In [2]:
import datetime
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport 
from gql.transport.exceptions import TransportQueryError
from operator import itemgetter
import regex as re
from collections import Counter, defaultdict
import json
import time
import csv
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from gql import gql, Client



In [2]:
# List tenants for connections

tenants = ['12th-gate-napa-valley', '1967-cellars-napa-valley', '1883-reserve-napa-valley',
       '510-pearl-napa-valley', '90-broad-napa-valley',
       'black-and-gold-napa-valley', 'bold-stripes-napa-valley',
       'brownstown-napa-valley', 'ever-blooming-napa-valley',
       'excelsior-napa-valley', 'green-19-napa-valley', 'fairwinds-estate-winery',
       'first-moon-napa-valley', 'foolish-club-napa-valley',
       'founders-rock-napa-valley', 'four-points-napa-valley',
       'golden-anchor-napa-valley', 'great-day-napa-valley',
       'great-minds-napa-valley', 'horns-tail-napa-valley',
       'in-hoc-napa-valley', 'killarney-napa-valley',
       'landfall-napa-valley', 'les-soeurs-fideles-napa-valley','long-gray-line-napa-valley','maverick-moon-napa-valley',
       'mountain-born-napa-valley', 'mystic-circle-napa-valley',
       'nineteen-seventeen-napa-valley', 'obbligato-napa-valley',
       'one-heart-napa-valley', 'parasol-napa-valley', 'perpetual-napa-valley',
       'phiphika-napa-valley', 'prowling-vineyards-napa-valley',
       'radiant-napa-valley', 'rattlesnake-napa-valley',
       'red-door-napa-valley', 'ruby-points-napa-valley',
       'seeing-eye-napa-valley', 'spindletop-napa-valley',
       'steel-curtain-napa-valley', 'tg-napa-valley', 'the-fleur-napa-valley',
       'the-ivy-napa-valley', 'three-degrees-napa-valley',
       'three-strings-napa-valley', 'tower-napa-valley',
       'true-north-napa-valley', 'unmatched-napa-valley',
       'vine-04-napa-valley', 'winning-drive-napa-valley']

In [1]:
tenants = ['510-pearl-napa-valley',"90-broad-napa-valley",'1967-cellars-napa-valley']

In [3]:
# Prueba descarga de info de un tenant
orders = []
#runs = 0

  
tenant_info = {
    'fairwinds-estate-winery': ('FEW', 'Fairwinds Estate Winery'),
    '12th-gate-napa-valley': ('12G', '12th Gate'),
    '510-pearl-napa-valley': ('5P', '510 Pearl'),
    '90-broad-napa-valley': ('90B', '90 Broad'),
    'black-and-gold-napa-valley': ('BG', 'Black & Gold'),
    'brownstown-napa-valley': ('BT', 'Brownstown'),
    'excelsior-napa-valley': ('EX', 'Excelsior'),
    'first-moon-napa-valley': ('FM', 'First Moon'),
    'foolish-club-napa-valley': ('FOO', 'Foolish Club'),
    'founders-rock-napa-valley': ('FR', 'Founders Rock'),
    'four-points-napa-valley': ('FP', 'Four Points'),
    'golden-anchor-napa-valley': ('GA', 'Golden Anchor'),
    'great-day-napa-valley': ('GD', 'Great Day'),
    'in-hoc-napa-valley': ('IH', 'In Hoc'),
    'killarney-napa-valley': ('KI', 'Killarney'),
    'maverick-moon-napa-valley': ('MM', 'Maverick Moon'),
    'mountain-born-napa-valley': ('MB', 'Mountain Born'),
    'mystic-circle-napa-valley': ('MC', 'Mystic Circle'),
    'nineteen-seventeen-napa-valley': ('NS', 'Nineteen Seventeen'),
    'one-heart-napa-valley': ('OH', 'One Heart'),
    'parasol-napa-valley': ('PR', 'Parasol'),
    'phiphika-napa-valley': ('PHI', 'PhiPhiKa'),
    'prowling-vineyards-napa-valley': ('PV', 'Prowling Vineyards'),
    'radiant-napa-valley': ('RA', 'Radiant'),
    'rattlesnake-napa-valley': ('RS', 'Rattlesnake'),
    'red-door-napa-valley': ('RD', 'Red Door'),
    'rituelle-napa-valley': ('RT', 'Rituelle'),
    'ruby-points-napa-valley': ('RP', 'Ruby Points'),
    'seeing-eye-napa-valley': ('SE', 'Seeing Eye'),
    'spindletop-napa-valley': ('ST', 'Spindletop'),
    'tg-napa-valley': ('TG', 'TG'),
    'the-ivy-napa-valley': ('IV', 'The Ivy'),
    'three-degrees-napa-valley': ('TD', 'Three Degrees'),
    'three-strings-napa-valley': ('TS', 'Three Strings'),
    'tower-napa-valley': ('TW', 'Tower'),
    'true-north-napa-valley': ('TN', 'True North'),
    'unmatched-napa-valley': ('UN', 'Unmatched'),
    'winning-drive-napa-valley': ('WD', 'Winning Drive'),
    'landfall-napa-valley': ('LF', 'Landfall'),
    'obbligato-napa-valley': ('OB', 'Obbligato'),
    'ever-blooming-napa-valley': ('EB', 'Ever Blooming'),
    'great-minds-napa-valley': ('GM', 'Great Minds'),
    'steel-curtain-napa-valley': ('SC', 'Steel Curtain'),
    '1967-cellars-napa-valley': ('CL', '1967 Cellars'),
    'horns-tail-napa-valley': ('HT', 'Horns & Tail'),
    'vine-04-napa-valley': ('V4', 'Vine 04'),
    'bold-stripes-napa-valley': ('BS', 'Bold Stripes'),
    'perpetual-napa-valley': ('PER', 'Perpetual'),
    'green-19-napa-valley': ('WS', 'Winning Standard'),
    '1883-reserve-napa-valley': ('YS', 'Yellowstone'),
    'long-gray-line-napa-valley': ('LG', 'Long Gray Line'),
    'les-soeurs-fideles-napa-valley': ('LS', 'Les Soeurs Fideles'),
    'the-fleur-napa-valley': ('TF', 'The Fleur')
}

for t in tenants: 
    tenantcode, brand = tenant_info.get(t, ('NaN', 'NaN'))


    params = {'cursor': 'start'}

    print(t)

    while True:
        url = 'https://api.commerce7.com/v1/order?orderPaidDate=gt:2023-04-10'
        response = requests.get(url, auth = HTTPBasicAuth('wilfredo@fewinery.com', 'Conexion@4$'), 
                                headers = {"tenant":t, "Content-Type":"application/json"},
                                params=params)

        res = response.json()
        #print(res)
        for row in res['orders']:
            row['tenant'] = t
            row['tenantCode'] = tenantcode
            row['uniqueOrderNumber'] = tenantcode + str(row['orderNumber'])
            orders.append(row)

        if (cursor := res.get('cursor')):
            params['cursor'] = cursor
            #print(len(res['orders']), tenantorders[-1]['orderNumber'], tenantorders[-1]['tenantCode'])
        else:
            #print(len(res['orders']), tenantorders[-1]['orderNumber'], tenantorders[-1]['tenantCode'])
            break

    #print(len(tenantorders), tenantorders[-1]['orderNumber'], tenantorders[-1]['tenantCode'])

    #for i in tenantorders:
    #    orders.append(i)
    
#    print(len(orders), orders[-1]['tenantCode'])

#print("finished")

#c = Counter(order['tenant'] for order in orders)
#print(dict(c))


510-pearl-napa-valley
90-broad-napa-valley
1967-cellars-napa-valley


In [4]:
len(orders)

12

In [5]:
errlist = []
orderlist = []

for payload in orders:    

    abbtost = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut',
            'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana',
            'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
            'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
            'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota',
            'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
            'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
            'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia', 'AS': 'American Samoa', 'GU': 'Guam', 'MP': 'Northern Mariana Islands',
            'PR': 'Puerto Rico', 'UM': 'United States Minor Outlying Islands', 'VI': 'U.S. Virgin Islands'}

    orderNumber = payload['uniqueOrderNumber']
    orderDate = payload['orderSubmittedDate']#.strftime('%Y-%m-%d')
    print(f"date : {orderDate}")
    #orderNumber = tenantcode+str(orderNumber)
    purchaseType = payload['purchaseType']
    #channel = payload['channel'] 
    #taxTotal = payload['taxTotal'] 
    #subTotal = payload['subTotal']
    #shipTotal = payload['shipTotal']
    #total = payload['total'] 
    giftMessage = "" if payload['giftMessage'] == None else str(payload['giftMessage'].replace('\r','').replace('\n', '-').replace('\"', '\\"'))
    orderDate = payload['orderSubmittedDate']

    """req_body['uniqueOrderNumber'] = orderNumber
    
    orderjson = json.dumps(req_body)
"""
    shiptotal = 0 if payload['shipping'] in [[], None, ""] else payload['shipping'][0]['price']/100 + payload['shipping'][0]['tax']/100

    firststr = """
mutation { 
  order_create( 
    data: {
      """
        
    carrier = "Generic" if payload['orderDeliveryMethod'] in ["Carry Out", "Pickup"] else payload['shipping'][0]['carrier'] 
    service = "" if payload['orderDeliveryMethod'] in ["Carry Out", "Pickup"] else "2 Day" if payload['shipping'][0]['service'] == "2 Day Air" else payload['shipping'][0]['service']
    title = carrier + " " + service
    fulfillmentStatus = "Express" if service in {"Standard Overnight", "2 Day"} else "pending"

    orderdata = """order_number: "%s" 
      shop_name: "Automatic" 
      fulfillment_status: "%s" 
      order_date: "%s" 
      packing_note: "%s" 
      shipping_lines:{ 
        title: "%s" 
        price: "%f" 
        carrier: "%s" 
        method: "%s" 
        } 
      """  % (orderNumber, fulfillmentStatus, orderDate, giftMessage, title, shiptotal, carrier, service)

    shipheader = """shipping_address: {
        """
    if  payload['orderDeliveryMethod'] in ["Carry Out", "Pickup"]:
        shipfname = payload['pickupBy']['firstName'] if payload['billTo'] is None else payload['billTo']['firstName']
        shiplname = payload['pickupBy']['lastName'] if payload['billTo'] is None else payload['billTo']['lastName']
        shipcompany = payload['pickupBy']['company'] if payload['billTo'] is None else "" if payload['billTo']['company'] == None else payload['billTo']['company']
        shipaddress1 = payload['pickupBy']['address'] if payload['billTo'] is None else payload['billTo']['address']
        shipaddress2 = payload['pickupBy']['address2'] if payload['billTo'] is None else "" if payload['billTo']['address2'] == None else payload['billTo']['address2']
        #billaddress2 = "" if billaddress2 == None else billaddress2
        shipcity = payload['pickupBy']['city'] if payload['billTo'] is None else payload['billTo']['city']
        shipstateCode = payload['pickupBy']['stateCode'] if payload['billTo'] is None else payload['billTo']['stateCode']
        shipstate = abbtost.get(shipstateCode)
        shipzipCode = payload['pickupBy']['zipCode'] if payload['billTo'] is None else payload['billTo']['zipCode']
        shipcountryCode = payload['pickupBy']['countryCode'] if payload['billTo'] is None else payload['billTo']['countryCode']
        shipemail = payload['customer']['emails'][0]['email']
        shipphone = payload['pickupBy']['phone'] if payload['billTo'] is None else payload['billTo']['phone']

    else:
        shipfname = payload['shipTo']['firstName']
        shiplname = payload['shipTo']['lastName']
        shipcompany = "" if payload['shipTo']['company'] == None else payload['shipTo']['company']
        shipaddress1 = payload['shipTo']['address']
        shipaddress2 = "" if payload['shipTo']['address2'] == None else payload['shipTo']['address2']
        shipcity = payload['shipTo']['city']
        shipstateCode = payload['shipTo']['stateCode']
        shipstate = abbtost.get(shipstateCode)
        shipzipCode = payload['shipTo']['zipCode']
        shipcountryCode = payload['shipTo']['countryCode']
        shipemail = payload['customer']['emails'][0]['email']
        shipphone = payload['shipTo']['phone']

        shipstr = """first_name: "%s" 
        last_name: "%s" 
        company: "%s" 
        address1: "%s" 
        address2: "%s" 
        city: "%s" 
        state: "%s"
        state_code: "%s" 
        zip: "%s" 
        country: "%s" 
        country_code: "%s" 
        email: "%s" 
        phone: "%s" 
        } 
      """ % (shipfname, shiplname, shipcompany, shipaddress1, shipaddress2, shipcity, shipstate, shipstateCode, shipzipCode, shipcountryCode, shipcountryCode, shipemail, shipphone)

        #  

    billheader = """billing_address:{ 
        """
    if payload['billTo'] == None: 
        billfname = shipfname 
        billlname = shiplname 
        billcompany = shipcompany 
        billaddress1 = shipaddress1 
        billaddress2 = shipaddress2 
        #billaddress2 = "" if billaddress2 == None else billaddress2
        billcity = shipcity 
        billstateCode = shipstateCode 
        billstate = shipstate 
        billzipCode = shipzipCode 
        billcountryCode = shipcountryCode 
        billemail = payload['customer']['emails'][0]['email']
        billphone = shipphone 

    else:
        billfname = payload['billTo']['firstName']
        billlname = payload['billTo']['lastName']
        billcompany = "" if payload['billTo']['company'] == None else payload['billTo']['company']
        billaddress1 = payload['billTo']['address']
        billaddress2 = "" if payload['billTo']['address2'] == None else payload['billTo']['address2']
        #billaddress2 = "" if billaddress2 == None else billaddress2
        billcity = payload['billTo']['city']
        billstateCode = payload['billTo']['stateCode']
        billstate = abbtost.get(billstateCode)
        billzipCode = payload['billTo']['zipCode']
        billcountryCode = payload['billTo']['countryCode']
        billemail = payload['customer']['emails'][0]['email']
        billphone = payload['billTo']['phone']

    billstr = """first_name: "%s" 
        last_name: "%s" 
        company: "%s" 
        address1: "%s" 
        address2: "%s" 
        city: "%s" 
        state: "%s"
        state_code: "%s" 
        zip: "%s" 
        country: "%s" 
        country_code: "%s" 
        email: "%s" 
        phone: "%s"
        } 
      """ %(billfname, billlname, billcompany, billaddress1, billaddress2, billcity, billstate, billstateCode, billzipCode, billcountryCode, billcountryCode, billemail, billphone)

        # 

    itemshdr = """line_items:"""

    prods = payload['items']

    hollidayPack = []

    for i in prods:
        hp = True if re.search(r"PC-\w{2,3}HD22", i['sku']) else False
        hollidayPack.append(hp)
        if i['sku'] == 'NW-WDWKFF':## Para arreglar el sku NW-WDWKFF
            i['sku'] = 'NW-WDWK'
        elif i['sku'] == 'TNWINEKEYF':
            i['sku'] = 'NW-TNWK'
        elif re.search(r'^NW', i['sku']) and re.search('WK', i['sku']) and i['sku'][-1] =='F':
            i['sku'] = re.sub(r"F$", "", i['sku']) #re.sub(r'F', "", sku)

    sku = itemgetter("sku")
    q_p_t = itemgetter("quantity", "price", "productTitle")

    res = defaultdict(list)
    for record in prods:
        res[sku(record)].append(q_p_t(record))

    prodarray = []
    for sku, values in res.items():
        quants, prices, productTitle = zip(*values)
        prodarray.append({"sku": sku, "quantity":sum(quants), "price":float(max(prices)), "productTitle":productTitle})


    itemsarray = []
    for item in prodarray:
        sku = item['sku']
        
        prodname = item['productTitle']
        price = item['price']/100
        quantity = item['quantity']
        pliid = orderNumber+"-"+sku
        itemstring = """{sku:"%s"   quantity:%d   price:"%f"  partner_line_item_id:"%s"   fulfillment_status:"pending"    quantity_pending_fulfillment:%d } """ % (sku, quantity, price, pliid, quantity)
        itemsarray.append(itemstring)

    # Talvez funciona agregando el } despues de los items, si no talvez hay que ingresar fecha u otro campo
    reqshipdate = datetime.datetime.strptime(orderDate, "%Y-%m-%dT%H:%M:%S.%fZ") + datetime.timedelta(days = 7)
    reqshipdate = reqshipdate.strftime("%Y-%m-%d")
    
    endstr = """
      required_ship_date:"%s" 
      }
      )
      {
    request_id
    complexity
    order {
      id
      order_number
    }
  }
} """ % reqshipdate

    itemsarray = str(itemsarray).replace("'","")
    #itemsarray = str(itemsarray).replace("\n","")
    orderstring = firststr+orderdata+shipheader+shipstr+billheader+billstr+itemshdr+str(itemsarray)+endstr
    #orderstring = orderstring.replace("'", "")

    #Enviar Mutation al GQL de Ship Hero


    token = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlJUQXlOVU13T0Rrd09ETXhSVVZDUXpBNU5rSkVOVVUxUmtNeU1URTRNMEkzTWpnd05ERkdNdyJ9.eyJodHRwOi8vc2hpcGhlcm8tcHVibGljLWFwaS91c2VyaW5mbyI6eyJuYW1lIjoid3Jvc2FsZXNAb3NpbmVhcnNob3JlLmNvbSIsIm5pY2tuYW1lIjoid3Jvc2FsZXMiLCJwaWN0dXJlIjoiaHR0cHM6Ly9zLmdyYXZhdGFyLmNvbS9hdmF0YXIvYmUwNTY0ZTk0OGQ5NTgxOTE2Zjg1MTRmMmM5MzU1MjI_cz00ODAmcj1wZyZkPWh0dHBzJTNBJTJGJTJGY2RuLmF1dGgwLmNvbSUyRmF2YXRhcnMlMkZ3ci5wbmciLCJhY2NvdW50X2lkIjoiNjU4NTgiLCJpc19hY2NvdW50X2FkbWluIjpmYWxzZX0sImlzcyI6Imh0dHBzOi8vbG9naW4uc2hpcGhlcm8uY29tLyIsInN1YiI6ImF1dGgwfDIyODI2MiIsImF1ZCI6WyJzaGlwaGVyby1wdWJsaWMtYXBpIl0sImlhdCI6MTY4MDYzMTk3MywiZXhwIjoxNjgzMDUxMTczLCJhenAiOiJtdGNid3FJMnI2MTNEY09OM0RiVWFITHFRelE0ZGtobiIsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgdmlldzpwcm9kdWN0cyBjaGFuZ2U6cHJvZHVjdHMgdmlldzpvcmRlcnMgY2hhbmdlOm9yZGVycyB2aWV3OnB1cmNoYXNlX29yZGVycyBjaGFuZ2U6cHVyY2hhc2Vfb3JkZXJzIHZpZXc6c2hpcG1lbnRzIGNoYW5nZTpzaGlwbWVudHMgdmlldzpyZXR1cm5zIGNoYW5nZTpyZXR1cm5zIHZpZXc6d2FyZWhvdXNlX3Byb2R1Y3RzIGNoYW5nZTp3YXJlaG91c2VfcHJvZHVjdHMgdmlldzpwaWNraW5nX3N0YXRzIHZpZXc6cGFja2luZ19zdGF0cyBvZmZsaW5lX2FjY2VzcyIsImd0eSI6WyJyZWZyZXNoX3Rva2VuIiwicGFzc3dvcmQiXX0.M0yWTtBYdPRZbonjd9oSHKuWX5ylJ9hGbrdi307Q-97ut4onrO4AksEe8GUrsD6AgWrXo2cRiDzqA-pDLhUo-HGxl9wc8NogMBjPA8YNAWg2qCVgyjxbPJ5umtsucG0534_00wMtUOjAWp10ifUBVRSHLO23-mc32xkYxvl3CrYcfb4NPo6YBPDKCNLufR9U3Es90kWlUKrx6TEmU0U65-Hz-nWdF96rQE2O3OeGWdmSnVPgr_0gPCroVaALSvUOozuqiJQT4eq52lfRLXtjj02inN7nxLApSIfx6hJH5Vk-hwrYqh_BgcnaylAI3PfKT7BdjWYucdp3oNo8Gn3KUA"
    #token = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlJUQXlOVU13T0Rrd09ETXhSVVZDUXpBNU5rSkVOVVUxUmtNeU1URTRNMEkzTWpnd05ERkdNdyJ9.eyJodHRwOi8vc2hpcGhlcm8tcHVibGljLWFwaS91c2VyaW5mbyI6eyJuYW1lIjoid3Jvc2FsZXNAb3NpbmVhcnNob3JlLmNvbSIsIm5pY2tuYW1lIjoid3Jvc2FsZXMiLCJwaWN0dXJlIjoiaHR0cHM6Ly9zLmdyYXZhdGFyLmNvbS9hdmF0YXIvYmUwNTY0ZTk0OGQ5NTgxOTE2Zjg1MTRmMmM5MzU1MjI_cz00ODAmcj1wZyZkPWh0dHBzJTNBJTJGJTJGY2RuLmF1dGgwLmNvbSUyRmF2YXRhcnMlMkZ3ci5wbmciLCJhY2NvdW50X2lkIjoiNjU4NTgiLCJpc19hY2NvdW50X2FkbWluIjpmYWxzZX0sImlzcyI6Imh0dHBzOi8vbG9naW4uc2hpcGhlcm8uY29tLyIsInN1YiI6ImF1dGgwfDIyODI2MiIsImF1ZCI6WyJzaGlwaGVyby1wdWJsaWMtYXBpIl0sImlhdCI6MTY2MTI4NzUyNywiZXhwIjoxNjYzNzA2NzI3LCJhenAiOiJtdGNid3FJMnI2MTNEY09OM0RiVWFITHFRelE0ZGtobiIsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgdmlldzpwcm9kdWN0cyBjaGFuZ2U6cHJvZHVjdHMgdmlldzpvcmRlcnMgY2hhbmdlOm9yZGVycyB2aWV3OnB1cmNoYXNlX29yZGVycyBjaGFuZ2U6cHVyY2hhc2Vfb3JkZXJzIHZpZXc6c2hpcG1lbnRzIGNoYW5nZTpzaGlwbWVudHMgdmlldzpyZXR1cm5zIGNoYW5nZTpyZXR1cm5zIHZpZXc6d2FyZWhvdXNlX3Byb2R1Y3RzIGNoYW5nZTp3YXJlaG91c2VfcHJvZHVjdHMgdmlldzpwaWNraW5nX3N0YXRzIHZpZXc6cGFja2luZ19zdGF0cyBvZmZsaW5lX2FjY2VzcyIsImd0eSI6WyJyZWZyZXNoX3Rva2VuIiwicGFzc3dvcmQiXX0.Y5386VI6T4l6cCw42VUTBGWQHBwSefoFj4Dwb2BBSnfeFyHnfbjaxrQCqDXoXCB74HUXYdSWXjPg86bpaWpUUHHqTWB2LRzcVwd2yAunc9A9g8UghoQkSC3p4GJR1i4vday9IZDVsiqSosw9yaf1vh4rhnC-XfDeptONtdw2Y_DPve2y2lyP6otC8rCbQLFbkKtozBCF5HMTGLev_aGplXYiuC2zswgXGMlFyJcnHOxh88DohkDnvrmaJUr4xS1UGR_mQHsLwGqmMQ-jtEw1IsQ86DlW3oi_uy4NkyQqmDF9D85PAINN6zQ78Bb54JKaMmnPc9i-5HKep-RdbkCKuA"

    #logging.info("itemnums:{}".format(len(payload['items'])))
    #logging.info("sku:{}".format(sku))

    #req_body['hollidayPack'] = "Gift" if sum(hollidayPack) > 0 else "Normal"

    if len(payload['items']) == 1 and sku == "50Deposit":
        print('Membership:%s' % orderNumber)
        #req_body['typeOfTransaction'] = 'Membership'
        #orderjson = json.dumps(req_body)
        #return func.HttpResponse(orderjson)
    
    elif purchaseType != 'Refund':
        _transport = RequestsHTTPTransport( url='https://public-api.shiphero.com/graphql', use_json=True, )
        _transport.headers = {"Authorization": "Bearer {}".format(token), "content-type": "application/json", }
        client = Client( transport=_transport, fetch_schema_from_transport=True, )
        #print(orderstring)
        query = gql(orderstring)
        
        

        try:
            #print('here')
            res = client.execute(query)
            time.sleep(0.5)
            print('Purchase:%s' % orderNumber)
            orderlist.append(orderNumber)

          #req_body['typeOfTransaction'] = 'Order'
        #orderjson = json.dumps(req_body)
        #return func.HttpResponse(orderjson)
        except TransportQueryError as e:
            error_message = e.args[0]
            errlist.append((orderNumber, error_message))
            print(error_message, orderNumber)

            
        except Exception as e:
            error_message = e.message
            errlist.append((orderNumber, error_message))
            print(error_message, 'orderNumber')
            print('error:%s' % orderNumber)
        

            
    

date : 2023-04-10T07:08:19.695Z
{'message': "A order for the shop 'Automatic' with order number 'CL1057' already exists", 'operation': 'order_create', 'field': 'order_create', 'request_id': '64342dd6afe4a175418ad6d1', 'code': 6} CL1057
date : 2023-04-10T07:08:20.242Z
{'message': "A order for the shop 'Automatic' with order number 'CL1061' already exists", 'operation': 'order_create', 'field': 'order_create', 'request_id': '64342dd7cf3dbe3c6cedc0a6', 'code': 6} CL1061
date : 2023-04-10T07:08:19.887Z
{'message': "A order for the shop 'Automatic' with order number 'CL1058' already exists", 'operation': 'order_create', 'field': 'order_create', 'request_id': '64342dd82316b9c47addc8ec', 'code': 6} CL1058
date : 2023-04-10T07:08:20.030Z
{'message': "A order for the shop 'Automatic' with order number 'CL1060' already exists", 'operation': 'order_create', 'field': 'order_create', 'request_id': '64342dd9a4d62e924e1fbc67', 'code': 6} CL1060
date : 2023-04-10T07:08:20.312Z
{'message': "A order for

In [6]:
orderlist

[]

In [75]:
# orderlist.append("CL1057")
orderlist_df =  pd.DataFrame(orderlist,columns=["order_number"])
orderlist_df

,order_number
0,CL1057
1,CL1057


In [50]:
len(errlist)

12

In [68]:
# errlist.append(("LL7777", "this is a testing error message error_message"))
errlist.pop()

('LL7777', 'this is a testing error message error_message')

In [69]:
errordf = pd.DataFrame(errlist, columns=['order', 'msg'])
errordf.head()

,order,msg
0,CL1057,"{'message': ""A order for the shop 'Automatic' ..."
1,CL1061,"{'message': ""A order for the shop 'Automatic' ..."
2,CL1058,"{'message': ""A order for the shop 'Automatic' ..."
3,CL1060,"{'message': ""A order for the shop 'Automatic' ..."
4,CL1062,"{'message': ""A order for the shop 'Automatic' ..."


In [70]:
# errordf.to_excel('orderCreationErrors.xlsx', index=False)
errordf.to_csv('orderCreationErrors.csv', index=False)
errordf

,order,msg
0,CL1057,"{'message': ""A order for the shop 'Automatic' ..."
1,CL1061,"{'message': ""A order for the shop 'Automatic' ..."
2,CL1058,"{'message': ""A order for the shop 'Automatic' ..."
3,CL1060,"{'message': ""A order for the shop 'Automatic' ..."
4,CL1062,"{'message': ""A order for the shop 'Automatic' ..."
5,CL1053,"{'message': ""A order for the shop 'Automatic' ..."
6,CL1054,"{'message': ""A order for the shop 'Automatic' ..."
7,CL1055,"{'message': ""A order for the shop 'Automatic' ..."
8,CL1056,"{'message': ""A order for the shop 'Automatic' ..."
9,CL1059,"{'message': ""A order for the shop 'Automatic' ..."


In [73]:
# Filter errors with 'already exists' in the error message
# The ~ operator is used to negate the result so that we get the rows that do not contain the string.
filtered_df = errordf[~errordf['msg'].str.contains('already exists')]

# Write the filtered data to a new CSV file
# filtered_df.to_csv('filtered_error_file.csv', index=False)
filtered_df



,order,msg


In [31]:
num_rows, num_cols = filtered_df.shape
print(f'The DataFrame has {num_rows} rows and {num_cols} columns.')
# Check if the DataFrame has any rows
if filtered_df.empty:
    print('The DataFrame is empty.')
else:
    # Process the DataFrame
    pass

The DataFrame has 0 rows and 2 columns.
The DataFrame is empty.


In [76]:

# Define the dataframes
# Check if the DataFrame has any rows



# Define email message
sender = 'marlon@fewinery.com'
recipient = ['mlopezg@osinearshore.com', 'marlon@fewinery.com']
msg = MIMEMultipart()
msg['Subject'] = 'orders error'
msg['From'] = sender
msg['To'] = ', '.join(recipient)

body= ""
# Add message body
if orderlist_df.empty:
    body1 = 'No Orders Number detected in this  process '
    
else:
    body1 = 'list of orders numbers'
   
    orderlist_df = orderlist_df.to_csv(index=False)
     # Add the CSV files as attachments to the email message
    part1 = MIMEApplication(orderlist_df, Name='orderlist.csv')
    part1['Content-Disposition'] = 'attachment; filename="orderlist.csv"'
    msg.attach(part1)


if filtered_df.empty:
    body2  = 'No errors detected in this  process '
   
else:
    body2 = 'Attached files contain list of order errors '

    Error_list = filtered_df.to_csv(index=False)
    # Add the CSV files as attachments to the email message
    part2 = MIMEApplication(Error_list, Name='Error_list.csv')
    part2['Content-Disposition'] = 'attachment; filename="Error_list.csv"'
    msg.attach(part2)

body = body1 + "\n" + body2
msg.attach(MIMEText(body, 'plain'))
# msg.attach(MIMEText(body2, 'plain'))



# Send email
smtp=smtplib.SMTP('smtp.office365.com', 587)
smtp.starttls()
smtp.login(sender, 'Bag97417')
smtp.sendmail(sender, recipient, msg.as_string())
smtp.quit()


(221, b'2.0.0 Service closing transmission channel')

In [47]:
# Define email message
sender = 'marlon@fewinery.com'
recipient = ['mlopezg@osinearshore.com', 'marlon@fewinery.com']
msg = MIMEMultipart()
msg['Subject'] = 'orders error'
msg['From'] = sender
msg['To'] = ', '.join(recipient)

# Define message bodies
if orderlist_df.empty:
    body1 = 'No Orders Number detected in this  process '
else:
    body1 = 'List of orders numbers:\n' + orderlist_df.to_string(index=False)

if filtered_df.empty:
    body2  = 'No errors detected in this  process '
else:
    body2 = 'Attached files contain list of order errors.\n' + filtered_df.to_string(index=False)

# Attach message bodies to email message
msg.attach(MIMEText(body1, 'plain'))
msg.attach(MIMEText(body2, 'plain'))

# Attach CSV files to email message
if not orderlist_df.empty:
    orderlist_df = orderlist_df.to_csv(index=False)
    part1 = MIMEApplication(orderlist_df, Name='orderlist.csv')
    part1['Content-Disposition'] = 'attachment; filename="orderlist.csv"'
    msg.attach(part1)
if not filtered_df.empty:
    Error_list = filtered_df.to_csv(index=False)
    part2 = MIMEApplication(Error_list, Name='Error_list.csv')
    part2['Content-Disposition'] = 'attachment; filename="Error_list.csv"'
    msg.attach(part2)

# Send email
smtp=smtplib.SMTP('smtp.office365.com', 587)
smtp.starttls()
smtp.login(sender, 'Bag97417')
smtp.sendmail(sender, recipient, msg.as_string())
smtp.quit()


(221, b'2.0.0 Service closing transmission channel')